In [1]:
import json
import requests

# ================= 配置区域 =================
# 1. 指向本地 Ollama 接口
OLLAMA_BASE_URL = "http://192.168.1.10:11434/api/generate"
OLLAMA_MODEL_NAME = "qwen3:latest"  # 确保你的Ollama中存在这个模型

# ===========================================

def distill_knowledge_local(question):
    # 针对 Qwen 优化的 Prompt，它对中文指令跟随很好
    prompt = f"""
    你是一位逻辑严密的老师。请针对以下问题，生成用于教学的思维链（Chain of Thought）。
    
    问题：{question}
    
    请务必严格按照以下JSON格式直接返回，不要包含Markdown代码块标记（如 ```json ）：
    {{
        "instruction": "{question}",
        "rationale": "这里写下详细的、分步骤的推理过程，先分析再计算，逻辑要清晰。",
        "output": "最终的简短答案"
    }}
    """
    
    try:
        # 构造Ollama API请求参数
        payload = {
            "model": OLLAMA_MODEL_NAME,
            "prompt": prompt,
            "stream": False,
            "format": "json"  # 告诉Ollama返回JSON格式响应
        }
        
        # 发送POST请求到Ollama API
        response = requests.post(OLLAMA_BASE_URL, json=payload, timeout=30)
        response.raise_for_status()  # 检查HTTP错误
        
        # 解析响应内容
        content = response.json()
        
        if isinstance(content, dict) and "response" in content:
            clean_content = content["response"].replace("```json", "").replace("```", "").strip()
            try:
                data_json = json.loads(clean_content)
                return data_json
            except json.JSONDecodeError:
                print(f"❌ JSON解析失败，模型输出了非标准格式:\n{clean_content}\n")
                return None
        else:
            print(f"❌ 响应格式异常: {content}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"网络请求出错: {e}")
        return None
    except Exception as e:
        print(f"API调用出错: {e}")
        return None

# 测试运行：我们用几个数学逻辑题来测试 Qwen 的推理生成能力
seed_questions = [
    "如果你有3个苹果，吃掉了1个，又买来了5个，现在有几个？",
    "一个笼子里有鸡和兔，头有10个，脚有28只，鸡兔各多少？",
    "用Python写一个函数，判断一个数字是不是素数。"
]

dataset = []

print(f"正在使用本地模型 {OLLAMA_MODEL_NAME} 进行蒸馏...\n")

for q in seed_questions:
    print(f"正在处理: {q} . ..")
    content = distill_knowledge_local(q)
    
    if content:
        # 清洗数据：有时候模型会忍不住加 ```json ... ```，我们需要去掉
        try:
            data_json = content
            dataset.append(data_json)
            print("✅ 生成成功！")
            print(f"  --> 思维过程片段: {data_json['rationale'][:50]}. ..")
        except Exception as e:
            print(f"❌ 数据处理异常: {e}")
    else:
        print("❌ 生成失败，无返回内容。")

# 保存结果
if dataset:
    filename = "local_distill_data.jsonl"
    with open(filename, "w", encoding="utf-8") as f:
        for entry in dataset:
            final_entry = {
                "instruction": entry["instruction"],
                "input": "",
                "output": f"【思考过程】\n{entry['rationale']}\n\n【最终答案】\n{entry['output']}"
            }
            json.dump(final_entry, f, ensure_ascii=False)
            f.write("\n")
    print(f"\n🎉 任务完成！数据已保存为 {filename}，共 {len(dataset)} 条。")


正在使用本地模型 qwen3:latest 进行蒸馏...

正在处理: 如果你有3个苹果，吃掉了1个，又买来了5个，现在有几个？ . ..
网络请求出错: HTTPConnectionPool(host='192.168.1.10', port=11434): Max retries exceeded with url: /api/generate (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x10f72ed30>, 'Connection to 192.168.1.10 timed out. (connect timeout=30)'))
❌ 生成失败，无返回内容。
正在处理: 一个笼子里有鸡和兔，头有10个，脚有28只，鸡兔各多少？ . ..


KeyboardInterrupt: 

In [2]:
import torch
from transformers import AutoTokenizer
import json

# 1. 加载学生模型的 Tokenizer (假设你用的是 Qwen2.5-0.5B)
# 如果本地还没下载，它会自动下载
model_id = "Qwen/Qwen2.5-0.5B-Instruct" 
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# 确保 pad_token 存在
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def process_func(example):
    """
    将生成的 CoT 数据转换为模型训练格式
    Example 格式: {"instruction": "...", "input": "", "output": "【思考过程】..."}
    """
    MAX_LENGTH = 512 # 演示用，实际可调大
    
    instruction = example["instruction"]
    output = example["output"] # 这里包含了 Teacher 生成的思维链

    # Qwen 的标准对话模板构建
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": instruction},
        {"role": "assistant", "content": output}
    ]
    
    # 使用 apply_chat_template 自动处理 <|im_start|> 等特殊 token
    # 这一步非常关键，不同模型模板不同
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    # 转换为 ID
    input_ids = tokenizer(text + tokenizer.eos_token, return_tensors="pt")["input_ids"][0]
    
    # === 关键点：制作 Labels (Masking) ===
    # 我们不希望模型学习 "User" 说的话，只学习 "Assistant" 说的话
    # 所以要把 User 部分的 token 在 label 中设为 -100 (PyTorch 会忽略)
    
    labels = input_ids.clone()
    
    # 找到 "assistant" 回复开始的位置
    # Qwen 的模板通常是: ... <|im_start|>assistant\n
    # 我们简单粗暴地重新 tokenize "user" 部分来找长度（严谨的做法需更复杂匹配）
    
    user_messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": instruction}
    ]
    user_text = tokenizer.apply_chat_template(user_messages, tokenize=False, add_generation_prompt=True)
    user_ids = tokenizer(user_text, return_tensors="pt")["input_ids"][0]
    
    user_len = len(user_ids)
    
    # 将 User 部分的 Label 设为 -100
    labels[:user_len] = -100
    
    # 截断或填充
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
        
    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": torch.ones_like(input_ids) # 简单处理
    }

# === 测试运行 ===
# 读取我们刚才生成的 local_distill_data.jsonl 的第一条
with open("local_distill_data.jsonl", "r", encoding="utf-8") as f:
    first_line = json.loads(f.readline())

processed = process_func(first_line)

print(f"原始问题: {first_line['instruction']}")
print(f"Token 总长度: {len(processed['input_ids'])}")
print(f"Labels 前10个 (应该是-100): {processed['labels'][:10]}")
print(f"Labels 最后10个 (应该有值): {processed['labels'][-10:]}")

# 解码看看模型到底要在哪一部分计算 Loss
valid_labels = processed['labels'][processed['labels'] != -100]
print("\n=== 模型将学习的内容 (Teacher 的 CoT) ===")
print(tokenizer.decode(valid_labels))

FileNotFoundError: [Errno 2] No such file or directory: 'local_distill_data.jsonl'

In [ ]:
!pip install datasets
!pip install trl
!pip install tf-keras

In [ ]:
import transformers
import trl

print(transformers.__version__)
print(trl.__version__)

In [ ]:
# 1. 引入 PyTorch 深度学习框架
# 作用：这是底层基础框架，用于张量计算、自动求导以及管理 GPU/CPU 资源。
import torch
# 2. 从 datasets 库中引入 load_dataset 函数
# 作用：用于加载训练数据。支持从 Hugging Face Hub 在线加载，也支持加载本地的 JSON/CSV/Parquet 文件。
from datasets import load_dataset
# 3. 从 transformers 库中引入三个核心组件
# AutoTokenizer: 自动加载分词器，将文本转换成模型能读懂的数字（Token ID）。
# AutoModelForCausalLM: 自动加载因果语言模型（如 Llama, Qwen, GPT 等自回归模型）的结构和权重。
# TrainingArguments: 用于定义训练时的超参数（如学习率、Batch Size、Epoch数、保存路径等）。
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
# 4. 从 trl (Transformer Reinforcement Learning) 库中引入 SFT 训练器和数据整理器
# SFTTrainer: 专门用于监督微调（Supervised Fine-Tuning）的训练器，简化了流程。
# DataCollatorForCompletionOnlyLM: 一种特殊的数据整理器。在指令微调中，我们希望模型只学习“回答”部分，
# 而不计算“提问”部分的 Loss。这个工具可以自动把“提问”部分掩盖（Mask）掉。
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
# 5. 从 peft 库中引入 LoraConfig
# 作用：PEFT (Parameter-Efficient Fine-Tuning) 是参数高效微调库。
# LoraConfig 用于配置 LoRA (Low-Rank Adaptation) 算法的参数（如秩 rank, alpha 等），
# 让我们不需要全量微调模型，只需微调极少量的参数，大大降低显存需求。
from peft import LoraConfig

In [ ]:

# ================= 配置 =================
MODEL_ID = "Qwen/Qwen2.5-0.5B-Instruct" 
DATA_FILE = "math_dataset_with_answers.jsonl" 

# 用于指定训练完成后模型、日志和检查点（checkpoint）保存的文件夹路径。
# 后续在定义 TrainingArguments 时会用到这个变量。如果该文件夹不存在，训练脚本通常会自动创建它。
OUTPUT_DIR = "./qwen2.5_distilled_checkpoint"

# 1. 加载数据
# 使用 HuggingFace dataset 库直接读取 jsonl

# json, 指定数据的解析格式。这意味着 datasets 库会使用内置的 JSON 加载脚本去读取文件。除了 "json"，常用的还有 "csv", "parquet", "text" 等。
# data_files, 这个参数告诉函数去哪里找数据。支持传入单个文件路径，也支持传入文件列表。
# split, 指定加载数据后的划分标签。load_dataset 默认会返回一个 DatasetDict（包含 train/test/valid 等）。
# 设置 split="train" 后，它会直接返回一个单纯的 Dataset 对象，里面包含全量数据。
# 如果不加这一行，你后续调用数据时需要用 dataset["train"] 才能取到数据。
dataset = load_dataset("json", data_files=DATA_FILE, split="train")

# 2. 加载模型与Tokenizer
# 根据模型 ID 加载对应的分词器。分词器负责将文本切分并转换为数字 ID。

# MODEL_ID, 模型在 Hugging Face Hub 上的 ID（如 "Qwen/Qwen2.5-7B"）或者本地模型的存储路径。
# trust_remote_code=True, 非常重要。许多新模型（如 Qwen, ChatGLM 等）的分词逻辑并没有集成在官方的 transformers 库中，
# 而是放在了模型仓库里的 Python 文件（如 tokenization_qwen.py）中。
# 设置为 True, 表示你允许从该模型仓库下载并执行这些自定义的 Python 代码。如果不设置，加载新模型通常会报错。
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

# 检查分词器是否有“填充符”（pad_token），如果没有，就用“结束符”（eos_token）代替。
# 在批量训练（Batch Training）时，必须把长短不一的句子补齐成同样的长度（矩阵要求），这就需要用到 pad_token。
# 现在，很多现代大模型（如 Llama 2, Llama 3, Qwen 等）为了节省词表空间，默认没有定义 pad_token。
# 如果不手动指定，训练时会报错。将 eos_token（End of Sentence）作为填充符是一种通用的解决方案。
if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token

# 加载模型 (这里为了演示简单，没有用量化，0.5B 很小，显存够用)
# MODEL_ID，指定要加载的模型路径或 ID。
# device_map="auto"，智能显存分配。
#   这个参数会让 accelerate 库自动计算如何加载模型。
#   如果是一张卡，它会把模型全放进去。
#   如果是多张卡，它会自动切分模型层，均匀分配到不同 GPU 上。
#   如果显存不够，它甚至会将部分层卸载（Offload）到 CPU 内存甚至硬盘上。
# torch_dtype=torch.float16，精度设置。
#   默认情况，模型权重通常是 FP32（32位浮点数），占用显存大。
#   torch.float16 (FP16)，半精度浮点数。这可以将显存占用直接减半，且对推理和微调的精度影响很小。
#   注：如果是比较新的显卡（如 RTX 30/40系列，A100/H100），这里推荐用 torch.bfloat16，它的数值稳定性比 float16 更好，训练不容易发散。
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, 
    device_map="auto", 
    torch_dtype=torch.float16
)

Generating train split: 0 examples [00:00, ? examples/s]

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen2.5-0.5B-Instruct/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x21e4c460eb0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 0b0e7eb8-907e-4eff-a647-131488dccde9)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen2.5-0.5B-Instruct/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x21e4c461870>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: de853889-6e2e-4a3c-ad27-13ed07cce1ad)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct/res

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

d:\ProgramData\miniconda3\envs\yolov8\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Administrator\.cache\huggingface\hub\models--Qwen--Qwen2.5-0.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:

# 3. 设置 LoRA (可选，但推荐)
# 全参数微调0.5B也可以，但LoRA更快更稳
# LoRA不更新模型原本的几十亿参数，而是给模型中指定的层插入两个低秩小矩阵来训练。这样显存占用极低，且效果逼近全量微调。
# r=8，秩 (Rank)。
#   这是 LoRA 中最重要的参数。它决定了可训练参数的数量。数值越大，模型能学到的信息越多，但显存占用和计算量也越大。
#   通常设为 8, 16, 32 或 64。对于一般微调，8 或 16 是性价比最高的选择。
# lora_alpha=16，缩放系数 (Scaling Factor)。
#   类似于学习率放大器。LoRA 的权重更新会被乘以 alpha/r。
#   通常设置为 r 的 2倍（即 alpha=2r）是经验上的最佳实践。
# target_modules=[...]，目标模块。指定要对模型的哪些层应用 LoRA。
#   列表中的名字（如 q_proj）对应 Transformer 内部的线性层名称。
#   特别说明，对于 Qwen、Llama 等模型，官方建议对所有线性层（Attention 层的 Q,K,V,O 和 MLP 层的 Gate,Up,Down）都进行微调，效果远好于只微调 Attention 层。
# lora_dropout=0.05，Dropout 率。
#   在训练过程中随机丢弃 5% 的神经元连接，防止模型过拟合（死记硬背训练数据）。
# bias="none"，偏置项设置。
#   "none" 表示不训练原模型中的 Bias 参数。这是为了最大程度节省显存。也可以选 "all" 或 "lora_only"，但通常 "none" 就够了。
# task_type="CAUSAL_LM"，任务类型。
#   告诉 LoRA 我们在做一个标准的“因果语言模型”任务（即 GPT 式的文本生成）。
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # Qwen 全线性层微调效果最好
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 4. 格式化函数。这是一个回调函数，会被 SFTTrainer 自动调用。
# 它的目的是把数据集中的原始列（instruction, thinking, output）转换成模型能看懂的一个长字符串。

# 思维链融合，代码将 thinking（思维链/CoT）和 output（标准答案）拼在了一起。
# 这意味着我们在教模型在回答问题前先输出思考过程。这是目前 DeepSeek-R1 等推理模型的核心训练方式。

# example 参数：这是传入的一批数据（Batch），包含多条样本。
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        # 拼接 "思考过程" 和 "最终答案"
        output = "[思考过程]\n"+example["thinking"][i]+"\n\n[最终答案]\n"+example["output"][i] # 这里包含了 Teacher 生成的思维链
        # 构建 Qwen 格式文本
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": example['instruction'][i]},
            {"role": "assistant", "content": output} # 这里的 output 包含思维链
        ]
        # 应用分词器的聊天模板
        # 根据 Qwen 模型的要求，自动添加特殊标记（Special Tokens）。
        #   例如：它会把 messages 变成类似这样的字符串：
        #   <|im_start|>system\nYou are...<|im_end|><|im_start|>user\n问题...<|im_end|><|im_start|>assistant\n[思考过程]...
        #   tokenize=False：非常关键。我们这里只生成字符串，不转成数字 ID。因为 SFTTrainer 内部会统一做 Tokenization。
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        output_texts.append(text)
    return output_texts

# 5. 关键，这是指令微调（SFT）中最重要的一个组件。它的作用是Mask（掩盖）掉用户提问部分的 Loss。
# 这个神器会自动帮你把 User 的部分 mask 掉 (设为 -100)，只计算 Assistant 回复的 Loss

# 为什么需要它？
#   如果你不加这个，模型在训练时会同时计算“用户问题”和“AI回答”的损失。
#   这会导致模型通过背诵用户的问题来降低 Loss，而不是学习如何回答。
#   使用了这个 Collator 后，模型只会被惩罚回答得不对的地方，而不会被惩罚复述问题的地方。

# 参数详细说明：
#   response_template，分隔符。告诉 Collator 从哪里开始是“AI 的回答”。
#   <|im_start|>assistant\n：这是 Qwen 模型特有的特殊 Token 序列，标志着 Assistant 开始说话。
#   Collator 会在整段文本中寻找这个字符串，把这个字符串之前的所有 Token（System Prompt + User Prompt）的 Label 设为 -100（PyTorch 中忽略计算 Loss 的标记）。
# tokenizer=tokenizer，Collator 需要分词器来把 response_template 字符串转成 Token ID，以便在转换后的数据中进行匹配查找。
response_template = "<|im_start|>assistant\n" 
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# 6. 配置训练参数
# 作用，TrainingArguments 是 Hugging Face transformers 库中用于定义训练过程所有超参数（Hyperparameters）的核心类。
# 它负责告诉训练器（Trainer）“该怎么练”——比如练多快（学习率）、练多少次（Epoch）、占多少显存（Batch Size）、往哪里存（Output Dir）等。

# output_dir， 指定输出目录。
#   训练过程中生成的所有文件都会存放在这里。包括：
#       Checkpoints（检查点）：训练过程中的模型权重的保存。
#       Logs（日志）：Loss 的变化记录等。
#       Config（配置）：最终模型的配置文件。

# per_device_train_batch_size，单张显卡上的训练批次大小。
#   它决定了每次参数更新前，显卡一次性读入多少条数据进行计算。
#   数值含义：这里设为 8，意味着如果你的显存够大，每次会同时把 8 条数据塞进显卡计算 Loss。
#   显存影响：这个数值越大，训练越快，但显存占用越高。如果报 OOM（显存溢出）错误，通常第一个要改小的就是这个参数。

# gradient_accumulation_steps，梯度累积步数。
#   这是小显存跑大 Batch Size 的核心技巧。
#   原理：模型会进行 4 次前向传播（Forward Pass），每次算完 Loss 后不立即更新权重，而是把梯度攒起来。等凑够了 4 次，再一次性更新权重。
#   实际/等效 Batch Size：Total Batch Size=per_device_batch_size×gradient_accumulation_steps×GPU数量
#   在本例中：8×4×1=32
#   这意味着虽然单次只读 8 条数据，但在数学上等同于一次读了 32 条数据进行训练，从而保证了训练的稳定性。

# learning_rate， 学习率（Optimizer 的步长）。
# 决定了模型参数每次更新变化的幅度。
# 数值背景：2e-4（即 0.0002）是 LoRA 微调的经验值。
# 对比：如果是全量微调（Full Fine-tuning），学习率通常很小（如 1e-5 或 2e-5）。但因为 LoRA 训练参数很少，通常需要更大的学习率才能收敛。

# logging_steps，日志打印频率。
# 设置每训练多少步（Step）就在控制台打印一次当前的 Loss 和学习率。
# 设为 1 是因为数据集极小在大规模训练中，通常设置为 10 或 100，否则屏幕会被日志刷屏，且轻微影响性能。

# num_train_epochs=3，训练轮次，一个epoch意味着模型把整个训练集从头到尾跑一遍
# 设置为3是微调领域的“黄金标准”，通常1-3个epoch就能取得很好的效果。再多练可能会导致过拟合，失去泛华能力

# save_strategy="epoch", 模型保存的策略。
# 决定了什么时候保存一个Checkjpoint(存档)
# epoch, 表示每跑完一轮，就自动保存一次模型
# steps， 每隔N步保存一次（配合save_steps参数）
# no, 训练结速前不保存任何中间结果（省硬盘）

# report_to="none"，可视化报告工具集成。
# transformer库支持自动把训练数据上传到WandB（Weights & Biases), TensorBoard, MLflow等平台
# none， 表示关闭上传功能。
# 在本地开发、测试或者不想注册WandB账号时，设置为none可以避免很多登录报错和网络连接问题
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=8, 
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    num_train_epochs=3, 
    save_strategy="epoch", 
    report_to="none"
)

# 7. 开始训练

# SFTTrainer（Supervised Fine-Tuning Trainer）是 trl 库提供的核心类。
# 它的作用是将之前准备好的模型、数据、超参数、微调策略全部组装在一起，构建一个可以执行训练任务的对象。
# 相比于 Hugging Face 原生的 Trainer，SFTTrainer 做了很多针对指令微调（Instruction Tuning）的自动化工作（如自动分词、自动应用 LoRA、自动处理 Dataset 格式等）。
print("🚀 开始黑盒蒸馏训练 (SFT)...")
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

trainer.train()

# 8. 保存你的成果
print(f"🎉 训练完成！保存适配器到 {OUTPUT_DIR}")
trainer.save_model(OUTPUT_DIR)

🚀 开始黑盒蒸馏训练 (SFT)...


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: a55be25e-c36e-41c7-a789-b7b183ddd5c8)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
d:\ProgramData\miniconda3\envs\yolov8\lib\site-packages\peft\mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
d:\ProgramData\miniconda3\envs\yolov8\lib\site-packages\peft\tuners\tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/1012 [00:00<?, ? examples/s]

Applying formatting function to train dataset:   0%|          | 0/1012 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1012 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1012 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1012 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
1,1.437400
2,0.808600
3,0.675600
4,0.612200
5,0.538900
6,0.473800
7,0.528000
8,0.579300
9,0.466000
10,0.513800


🎉 训练完成！保存适配器到 ./qwen2.5_distilled_checkpoint


In [9]:
from peft import PeftModel

# 1. 准备测试问题 (用训练过的问题测试过拟合，用新问题测试泛化)
#test_question = "如果你有3个苹果，吃掉了1个，又买来了5个，现在有几个？" 
test_question = "三角形的底是8，高是3，求其面积是多少"
# 2. 定义推理函数
def run_inference(model, tokenizer, question):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": question}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.1) # 低温以便复现
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n=== ⚔️ 效果对决 ⚔️ ===\n")

# --- 回合 1: 原始傻瓜模型 (Base Student) ---
# 重新加载纯净模型
base_model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto", torch_dtype=torch.float16)
print(f"🔴 原始模型回答:\n{run_inference(base_model, tokenizer, test_question)}")

# --- 回合 2: 蒸馏后的模型 (Distilled Student) ---
# 加载刚才训练好的 LoRA
print("\nLoading LoRA adapters...")
distilled_model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
print(f"🟢 蒸馏模型回答:\n{run_inference(distilled_model, tokenizer, test_question)}")


=== ⚔️ 效果对决 ⚔️ ===

🔴 原始模型回答:
system
You are a helpful assistant.
user
三角形的底是8，高是3，求其面积是多少
assistant
三角形的面积可以通过公式计算得出。对于一个等腰三角形（即有两个相等的边），其面积公式为：

\[ \text{面积} = \frac{1}{2} \times \text{底} \times \text{高} \]

给定的三角形底是8，高是3，代入上述公式得：

\[ \text{面积} = \frac{1}{2} \times 8 \times 3 = 4 \times 3 = 12 \]

所以，这个三角形的面积是12平方单位。

Loading LoRA adapters...
🟢 蒸馏模型回答:
system
You are a helpful assistant.
user
三角形的底是8，高是3，求其面积是多少
assistant
[思考过程]
三角形的面积计算公式是底乘以高再除以2。首先，确定底边长度为8厘米，高为3厘米。然后，将底和高相乘得到8乘以3等于24。最后，将结果除以2，得到面积为12平方厘米。

[最终答案]
12
